<a href="https://www.kaggle.com/code/reaganodhiambootieno/netflix-data-wrangling?scriptVersionId=241625739" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


# **Data Wrangling Project**
Name: Reagan Odhiambo Otieno
Date: 24 May 2025

This project focuses on practicing data wrangling techniques using the Netflix Titles dataset from Kaggle. The goal is to clean, structure, and validate the data for further analysis or visualization.


# **Objectives**

1. Load and explore the dataset using pandas.
2. Identify and address data quality issues like missing values and duplicates.
3. Structure and enrich the dataset using transformation techniques.
4. Validate the final output to ensure it's clean, consistent, and usable.
5. Export the cleaned dataset as a CSV.


# **Import Data**

In [2]:
# Load the dataset
df = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')

# **Step 1: Discovery**

In [3]:
#Have a quick overview of the data
df.info()
# Number of rows and columns
print("Shape of the dataset (R x C):", df.shape)
# List of all column names
print("Columns in the dataset:\n", df.columns.tolist())
# Data types of each column
print("Data types:\n", df.dtypes)
# Group and Count of missing (null) values in each column
print("Missing values per column:\n", df.isnull().sum())
# Group and Count of duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
Shape of the dataset (R x C): (8807, 12)
Columns in the dataset:
 ['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 'rating', 'duration', 'listed_in', 'description']
Data types:
 show_id         object
type      

# **Step 2: Structuring**

In [4]:
# Convert date_added to datetime
df['date_added'] = pd.to_datetime(df['date_added'], format='mixed')

# Split 'duration' into numeric value and unit
df[['duration_value', 'duration_unit']] = df['duration'].str.extract(r'(\d+)\s*(\w+)')
df['duration_value'] = pd.to_numeric(df['duration_value'], errors='coerce')

# Preview
print(df[['duration_value', 'duration_unit']].head())

   duration_value duration_unit
0            90.0           min
1             2.0       Seasons
2             1.0        Season
3             1.0        Season
4             2.0       Seasons


# **Step 3: Ceaning**

In [5]:
# Check for duplicate rows
print("Duplicate rows before:", df.duplicated().sum())

# Drop duplicate rows
df = df.drop_duplicates()

# Drop description column because it will not be used
if 'description' in df.columns:
    df.drop(columns=['description'], inplace=True)

# Impute Director values by using relationship between cast and director

# List of Director-Cast pairs and the number of times they appear
df['dir_cast'] = df['director'] + '---' + df['cast']
counts = df['dir_cast'].value_counts() #counts unique values
filtered_counts = counts[counts >= 3] #checks if repeated 3 or more times
filtered_values = filtered_counts.index #gets the values i.e. names
lst_dir_cast = list(filtered_values) #convert to list

dict_direcast = dict()
for i in lst_dir_cast:
    director, cast = i.split('---')
    dict_direcast[director] = cast

for i in range(len(dict_direcast)): 
    df.loc[(df['director'].isna()) & (df['cast'] == list(dict_direcast.items())[i][1]), 'director'] = list(dict_direcast.items())[i][0]

# Assign Not Given to all other director fields
df.loc[df['director'].isna(), 'director'] = 'Not Given'

# Use directors to fill missing countries
directors = df['director']
countries = df['country']
# pair each director with their country use zip() to get an iterator of tuples
pairs = zip(directors, countries)
# Convert the list of tuples into a dictionary
dir_cntry = {k: v for k, v in pairs if pd.notna(k) and pd.notna(v)}  # Only include non-null pairs

# Director matched to Country values used to fill in null country values
for director, country in dir_cntry.items():    
    df.loc[(df['country'].isna()) & (df['director'] == director), 'country'] = country

# Assign Not Given to all other country fields
df.loc[df['country'].isna(), 'country'] = 'Not Given'

# Assign Not Given to all other fields
df.loc[df['cast'].isna(), 'cast'] = 'Not Given'

# dropping other row records that are null
df.drop(df[df['date_added'].isna()].index, axis=0, inplace=True)
df.drop(df[df['rating'].isna()].index, axis=0, inplace=True)
df.drop(df[df['duration'].isna()].index, axis=0, inplace=True)


Duplicate rows before: 0


# **Step 4: Error**

In [6]:
# Ensure 'date_added' is in datetime format (if not already)
df['date_added'] = pd.to_datetime(df['date_added'])

# Check if any 'date_added' year is before 'release_year'
invalid_dates = df['date_added'].dt.year < df['release_year']
print(f"Number of invalid records: {sum(invalid_dates)}")

# Display problematic records
print(df.loc[invalid_dates, ['date_added', 'release_year']])

# Sample some records to verify
print(df.iloc[[1551, 1696, 2920, 3168]])

# Confirm no more inconsistencies
print(f"Final count of invalid records: {sum(df['date_added'].dt.year < df['release_year'])}")



Number of invalid records: 14
     date_added  release_year
1551 2020-12-14          2021
1696 2020-11-15          2021
2920 2020-02-13          2021
3168 2019-12-06          2020
3287 2019-11-13          2020
3369 2019-10-25          2020
3433 2019-10-11          2020
4844 2018-05-30          2019
4845 2018-05-29          2019
5394 2017-07-01          2018
5658 2016-12-23          2018
5677 2016-12-13          2017
7063 2018-10-26          2019
7112 2013-03-31          2016
     show_id     type          title   director  \
1551   s1552  TV Show          Hilda  Not Given   
1696   s1697  TV Show   Polly Pocket  Not Given   
2920   s2921  TV Show  Love Is Blind  Not Given   
3168   s3169  TV Show   Fuller House  Not Given   

                                                   cast  \
1551  Bella Ramsey, Ameerah Falzon-Ojo, Oliver Nelso...   
1696  Emily Tennant, Shannon Chan-Kent, Kazumi Evans...   
2920                        Nick Lachey, Vanessa Lachey   
3168  Candace Cameron Bure, 

# **Step 5: Validation**

In [7]:
# 1. Convert to proper data types
df['date_added'] = pd.to_datetime(df['date_added'])
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
df['release_year'] = df['release_year'].astype(int)

# 2. Check for logical inconsistencies
# Date added before release year
date_issues = df[df['date_added'].dt.year < df['release_year']]
print(f"Records added before release year: {len(date_issues)}")

# Records before Netflix existed (1997)
ancient_records = df[df['date_added'].dt.year < 1997]
print(f"Records before 1997: {len(ancient_records)}")

# Invalid durations
invalid_durations = df[df['duration'] <= 0]
print(f"Invalid durations: {len(invalid_durations)}")

# 3. Remove temporary columns
df.drop(columns=['dir_cast'], inplace=True, errors='ignore')

# final check
# 1. Verify no missing values in key columns
print("\nMissing values check:")
print(df[['director', 'cast', 'country', 'date_added', 'rating', 'duration']].isnull().sum())

# 2. Data type verification
print("\nData types:")
print(df.dtypes)

# 3. Sample inspection
print("\nSample data:")
print(df.sample(5))

# 4. Reset index
df.reset_index(drop=True, inplace=True)

print("\nData cleaning and validation complete!")
print(f"Final dataset shape: {df.shape}")

Records added before release year: 14
Records before 1997: 0
Invalid durations: 0

Missing values check:
director         0
cast             0
country          0
date_added       0
rating           0
duration      8790
dtype: int64

Data types:
show_id                   object
type                      object
title                     object
director                  object
cast                      object
country                   object
date_added        datetime64[ns]
release_year               int64
rating                    object
duration                 float64
listed_in                 object
duration_value           float64
duration_unit             object
dtype: object

Sample data:
     show_id     type                                       title  \
3009   s3010    Movie                           A Fall from Grace   
6865   s6866    Movie                                 Golden Time   
232     s233  TV Show                            Wheel of Fortune   
390     s391  TV Show 

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


# **Publish**

In [8]:
# Save as CSV 
df.to_csv('/kaggle/working/cleaned_netflix.csv', index=False)